In [11]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [12]:
from sailor import RouteGenConfig, SailorDataWarehouse

context="ERP admin panel"

_config = RouteGenConfig.from_env()
_warehouse = SailorDataWarehouse(_config, context, db_path="../db")

routes = await _warehouse.create_routes(count=20)
sessions_group = await _warehouse.create_sessions(routes, count=200)

In [13]:
from typing import List
from sklearn.model_selection import train_test_split
from sailor.types.route_specs import SessionSpec

train_sessions: List[SessionSpec] = []
test_sessions: List[SessionSpec] = []

for _sessions in sessions_group:
    _train, _test = train_test_split(_sessions, shuffle=True, test_size=0.2, random_state=14)
    train_sessions.extend(_train)
    test_sessions.extend(_test)

print(f"Train sessions: {len(train_sessions)}")
print(f"Test sessions: {len(test_sessions)}")

Train sessions: 1607
Test sessions: 408


In [14]:
from sailor.sailor_engine import KNNSailorEngine, SVCSailorEngine

knn_engine = KNNSailorEngine()
knn_engine.fit(routes, train_sessions)

svc_engine = SVCSailorEngine()
svc_engine.fit(routes, train_sessions)

Pipeline(steps=[('tfidf', TfidfVectorizer(stop_words='english')),
                ('svc', LinearSVC(class_weight='balanced', max_iter=2000))])

In [15]:
from sailor.sailor_engine import SailorEngine
from sklearn.metrics import classification_report, top_k_accuracy_score

top_k_score = 5
test_query = "Create a new flight"

def pre_validate_model(model: SailorEngine, sessions: List[SessionSpec]):
    print(f"Validating {model.__class__.__name__}...")

    test_queries = [s.context for s in sessions]
    targets = model.documentor.transform([s.target for s in sessions])
    target_names = [
        model.documentor.get_route(id).path
        for id in model.documentor.labels_
    ]

    return test_queries, targets, target_names

In [16]:
_test_queries, _targets, _target_names = pre_validate_model(knn_engine, test_sessions)

_predictions = knn_engine.pipeline.predict(_test_queries)
print(classification_report(_targets, _predictions, target_names=_target_names, zero_division=0))

_predictions_proba = knn_engine.pipeline.predict_proba(_test_queries)
_top_k_accuracy = top_k_accuracy_score(_targets, _predictions_proba, k=top_k_score)
print(f"Top-{top_k_score} accuracy: {_top_k_accuracy:.2f}")

Validating KNNSailorEngine...
                          precision    recall  f1-score   support

         /admin/invoices       1.00      0.80      0.89        20
   /admin/users/:id/edit       0.62      0.95      0.75        21
  /admin/invoices/create       0.74      0.81      0.77        21
         /admin/products       0.81      0.62      0.70        21
        /admin/customers       0.81      0.62      0.70        21
 /admin/inventory/adjust       0.58      0.70      0.64        20
     /admin/users/create       0.84      0.76      0.80        21
          /admin/reports       0.87      0.65      0.74        20
 /admin/settings/general       0.83      0.50      0.62        20
            /admin/users       0.85      0.55      0.67        20
        /admin/inventory       0.59      0.50      0.54        20
         /admin/settings       0.60      0.75      0.67        20
    /admin/customers/:id       0.75      0.90      0.82        20
       /admin/orders/:id       0.83      1.00

In [17]:
routes = knn_engine.predict(test_query)[:top_k_score]
for route in routes:
  print(f"Path: {route.path}, Score: {route.score:.2f}")
  print(f"Context: {route.context}")

Path: /admin/products/create, Score: 0.25
Context: admin products create products create erp admin new product add item New product setup Add product New product set Creating product listing Add new product form Add new product Create product listing Add new product New product creation form Adding new product catalog Need create new product Adding new product sell Adding new good Add new item product catalog Add anew item New product creation Create prodcut Create new product Creating new item New product creation Add item product list Adding new product Create new product data Create new product sale Add new item product catalog New product listing Add new product Adding new item Create new product site add product New product Add product details Add new product listing Create new merchandise Need add new item Create new product webstore Create new itme Add new product Create new product entry Create new product new product
Path: /admin/users/create, Score: 0.23
Context: admin users 

In [28]:
_test_queries, _targets, _target_names = pre_validate_model(svc_engine, test_sessions)

_predictions = svc_engine.pipeline.predict(_test_queries)
report = classification_report(_targets, _predictions, target_names=_target_names, zero_division=0)
print(report)

_decisions = svc_engine.pipeline.decision_function(_test_queries)
_top_k_accuracy = top_k_accuracy_score(_targets, _decisions, k=top_k_score)

print(f"Top-{top_k_score} accuracy: {_top_k_accuracy:.2f}")

Validating SVCSailorEngine...
                          precision    recall  f1-score   support

         /admin/invoices       0.94      0.80      0.86        20
   /admin/users/:id/edit       0.61      0.90      0.73        21
  /admin/invoices/create       0.77      0.81      0.79        21
         /admin/products       0.81      0.62      0.70        21
        /admin/customers       0.85      0.52      0.65        21
 /admin/inventory/adjust       0.56      0.75      0.64        20
     /admin/users/create       0.94      0.71      0.81        21
          /admin/reports       0.88      0.70      0.78        20
 /admin/settings/general       0.77      0.50      0.61        20
            /admin/users       0.77      0.50      0.61        20
        /admin/inventory       0.60      0.45      0.51        20
         /admin/settings       0.60      0.75      0.67        20
    /admin/customers/:id       0.72      0.90      0.80        20
       /admin/orders/:id       0.83      1.00

In [19]:
routes = svc_engine.predict(test_query)[:top_k_score]
for route in routes:
  print(f"Path: {route.path}, Score: {route.score:.2f}")
  print(f"Context: {route.context}")

Path: /admin/products/create, Score: -0.22
Context: admin products create products create erp admin new product add item New product setup Add product New product set Creating product listing Add new product form Add new product Create product listing Add new product New product creation form Adding new product catalog Need create new product Adding new product sell Adding new good Add new item product catalog Add anew item New product creation Create prodcut Create new product Creating new item New product creation Add item product list Adding new product Create new product data Create new product sale Add new item product catalog New product listing Add new product Adding new item Create new product site add product New product Add product details Add new product listing Create new merchandise Need add new item Create new product webstore Create new itme Add new product Create new product entry Create new product new product
Path: /admin/users/create, Score: -0.40
Context: admin user

In [20]:
_models_path = "../build/models"
knn_engine_path = knn_engine.save_model("knn_model", _models_path)
print(f"Model saved to {knn_engine_path}, size: {os.path.getsize(knn_engine_path) / 1024:.2f} KB")

svc_engine_path = svc_engine.save_model("svc_model", _models_path)
print(f"Model saved to {svc_engine_path}, size: {os.path.getsize(svc_engine_path) / 1024:.2f} KB")

Model saved to ../build/models/knn_model.pkl, size: 440548.25 KB
Model saved to ../build/models/svc_model.pkl, size: 440637.93 KB
